CIFAR-10-CLASSIFCATION MODEL

In [ ]:
import tensorflow as tf
import torch as th
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def unpickle(file): # - http://www.cs.toronto.edu/~kriz/cifar.html - this will open the file and return a dictionary 
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
dataset = unpickle("cifar-10-batches-py\data_batch_1") # loading data_batch_1 into the variable dataset
metadata = unpickle(r"cifar-10-batches-py\batches.meta") # These are the label names

Instead of importing the dataset directly, I decided to install it as a zip from online, to practice loading and wrangling the data

The website source of the dataset states:

 **data** -- a 10000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.

**labels** -- a list of 10000 numbers in the range 0-9. The number at index i indicates the label of the ith image in the array data.


In [ ]:
print(dataset.keys(),
metadata.keys())

We can see the the relevant items in the dataset and metadata dictionary

In [ ]:
labels = dataset[b'labels']
data = dataset[b'data']

In [ ]:
len(data)
len(labels)

As you can see, there are 10000 images + labels in our dataset. I will split the dataset in 80-20 training-testing into the following variables:

In [ ]:
X_train = data[:8000]
X_test = data[8000:10000]

y_train = labels[:8000]
y_test = labels[8000:10000]

In [ ]:
metadata.keys()

In [ ]:
keys = metadata[b'label_names']
keys

a label of 0 should be a airplane, a label of 1 should be of a automobile

In [ ]:
y_train[0]
# plt.imshow(X_train[0]) # This line throws a TypeError: Invalid shape (3072,) for image data. I realized, this is actually a 32 x 32 size image with RGB, however since there is nothing delimiting the rows
# So it just shows as 3072 - The images seem to have flattened, meaning I would need to "unflattern" them if I want to view them
# https://stackoverflow.com/questions/36967920/numpy-flatten-rgb-image-array 


In [ ]:
test = X_train[0]

In [ ]:
reshaped_image = test.reshape(32,32,3) # 32 x 32 (image size) with the 3 denoting the RGB values

plt.imshow(reshaped_image)


I'm going to have to reformat the array If I want to view the image, I will create a function that will allow me to view the image without changing the data structure of the original image

In [ ]:
def show_image(index = 0, feature_data = X_train, label_data = y_train, keys = keys):
        """
        input feature, and label data, and the keys dictionary, optional param index

        displays the image + label
        """
        # The images are stored as a 3072 element vector, I need to reshape this into a tensor 
        # The image source states "The first 1024 bytes are the red channel values, the next 1024 the green,
        # and the final 1024 the blue. The values are stored in row-major order, so the first 32 bytes are the red channel values of the first row of the image."
        res = feature_data[index].reshape(3,32,32) 
        #matplotlib reads the RGB channels last in the matrix. So I'm rearranging the tensor accordingly
        res = np.transpose(res, axes=[1, 2, 0]) 

        plt.imshow(res)
        plt.title(keys[label_data[index]].decode("utf-8")) # adds the label; the decode removes the b and '' from the label for a pleasant viewing experience
        plt.axis('off') # I don't want the axis to show either
        plt.show()




In [ ]:
show_image()

Beautiful!

Creating model using keras

In [ ]:
X_train, X_test = X_train / 255.0, X_test / 255.0
# since the values range from 1 to 256, I want them to range from 0-1, i will divide by 255 to achieve this. 
# Sources state this allows the model to perform more efficiently


In [108]:
model_v1 = tf.keras.Sequential([ # My model should have one input and one output tensor
    tf.keras.layers.Dense(128, activation=tf.nn.relu, input_shape = (3072,)), # input shape of 3072
    tf.keras.layers.Dense(128, activation=tf.nn.relu), # relu is a activation function, which determines the
    # output shape of each node in the layer
    tf.keras.layers.Dense(10) ])


    
# There is 2 layers with 2 nodes each, and a output layer with 10 nodes, each representing one of the 
# 10 possible categories


    

Training the model

Defining loss and gradient functions

In [109]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss(model, x, y, training):
    y_ = model(x,training=training)

    return loss_object(y_true=y, y_pred=y_)


In [110]:
l = loss(model_v1, X_train, y_train, training=False)
print("Loss test: {}".format(l))

Loss test: 2.37630558013916


In [111]:
def grad(model,inputs,targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, training = True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

Creating optimizer 



In [112]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01)
loss_value, grads = grad(model_v1, X_train, y_train)
print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model_v1.trainable_variables))

print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
                                          loss(model_v1, X_train, y_train, training=True).numpy()))

Step: 0, Initial Loss: 2.37630558013916
Step: 1,         Loss: 2.3525595664978027


In [115]:
model_v1.compile(loss = l, optimizer = optimizer, metrics = ['accuracy'])

In [129]:
model_v1.fit(X_train,y_train,batch_size=32)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})

When trying to compile my model I run into this error

    ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})

I think I can fix this by converting my datasets into numpy arrays\


In [130]:
X_train_v2 = np.array(X_train)
y_train_v2 = np.array(y_train)
X_test_v2 = np.array(X_test)
y_test_v2 = np.array(y_test)

l = loss(model_v1, X_train_v2, y_train_v2, training=False)

optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01)
loss_value, grads = grad(model_v1, X_train_v2, y_train_v2)
optimizer.apply_gradients(zip(grads, model_v1.trainable_variables))



<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [131]:
model_v1.compile(loss = l, optimizer = optimizer, metrics = ['accuracy'])

In [132]:
model_v1.fit(X_train_v2,y_train_v2,batch_size=32)

ValueError: in user code:

    File "c:\Users\maste\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\maste\anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\maste\anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\maste\anaconda3\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\maste\anaconda3\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "c:\Users\maste\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 184, in __call__
        self.build(y_pred)
    File "c:\Users\maste\anaconda3\lib\site

Now I seem to get this error:

    ValueError: Could not interpret loss function identifier: 2.3097445964813232

I believe this is a problem with my custom loss function

In [140]:
model_v1.compile(loss = 'SparseCategoricalCrossentropy', optimizer = optimizer, metrics = ['accuracy'])

In [141]:
model_v1.fit(X_train_v2,y_train_v2,batch_size=32,epochs = 5, validation_split = 0.1)

Epoch 1/5
225/225 [==============================] - 2s 7ms/step - loss: 2.3026 - accuracy: 0.1088 - val_loss: 2.3026 - val_accuracy: 0.0962
Epoch 2/5
225/225 [==============================] - 1s 5ms/step - loss: 2.3026 - accuracy: 0.1088 - val_loss: 2.3026 - val_accuracy: 0.0962
Epoch 3/5
225/225 [==============================] - 1s 5ms/step - loss: 2.3026 - accuracy: 0.1088 - val_loss: 2.3026 - val_accuracy: 0.0962
Epoch 4/5
225/225 [==============================] - 1s 6ms/step - loss: 2.3026 - accuracy: 0.1088 - val_loss: 2.3026 - val_accuracy: 0.0962
Epoch 5/5
225/225 [==============================] - 1s 5ms/step - loss: 2.3026 - accuracy: 0.1088 - val_loss: 2.3026 - val_accuracy: 0.0962


The loss and accuracy are not changing between epochs, There seems to be a problem